In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn

plt.style.use('seaborn')
sns.set(font_scale=2.5) # 이 두줄은 본 필자가 항상 쓰는 방법입니다. matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore') # 워닝 메세지를 생략해 줍니다. 차후 버전관리를 위해 필요한 정보라고 생각하시면 주석처리 하시면 됩니다.
%matplotlib inline

In [61]:
df_train = pd.read_csv('./data/custom_contest/train.csv')
df_test = pd.read_csv('./data/custom_contest/test.csv')
df_train.head()
for i,col in enumerate(list(df_train.columns)[2:-3]):
    df_train[col].astype(str)
df_train.fillna('missing')

,신고번호,신고일자,통관지세관부호,신고인부호,수입자부호,해외거래처부호,특송업체부호,수입통관계획코드,수입신고구분코드,수입거래구분코드,...,운송수단유형코드,반입보세구역부호,HS10단위부호,적출국가코드,원산지국가코드,관세율구분코드,관세율,검사결과코드,우범여부,핵심적발
0,57298928,2020-01-01,121,2O5A2,82ZHWNL,missing,TQ18AK,D,B,15,...,10,2106003,8481900000,US,US,A,8.0,N3,1,1
1,85092852,2020-01-01,30,305K5,5IS70LE,missing,missing,C,B,11,...,40,4077010,2106909099,US,US,A,8.0,A,0,0
2,63014158,2020-01-01,20,CGMT6,GJ5KBL3,R9ZQOG7,missing,D,B,11,...,40,4077007,6307909000,US,US,A,10.0,A,0,0
3,40175917,2020-01-01,40,QWUTG,PBYW02T,missing,missing,C,B,94,...,40,4077106,6505009090,CN,CN,A,8.0,A,0,0
4,11602631,2020-01-01,30,0X1CO,MCX0GJB,4Z9PX0Y,missing,C,B,11,...,40,2006075,6204320000,CN,CN,FCN1,5.2,M1_N1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89350,78046993,2020-11-22,10,M9SYU,K8195QD,missing,PR5UFJ,C,B,11,...,10,4002001,703101000,CN,CN,W2,135.0,A,0,0
89351,85641293,2020-11-22,41,T7VQN,MSE4EAG,missing,missing,E,E,15,...,40,4077008,6804220000,CN,CN,FCN1,0.0,A,0,0
89352,15760215,2020-11-22,40,7Q31W,5BC6UCR,QW3LA8B,missing,C,B,29,...,10,2086001,8210005000,CN,CN,A,8.0,A,0,0
89353,20569033,2020-11-22,40,UJ0JR,U7KJPA6,TJW57CJ,O04TIW,F,B,15,...,40,1618003,3304999000,CA,CA,C,6.5,S1,1,2


In [62]:
df_train.isnull().sum() / df_train.shape[0]

신고번호        0.000000
신고일자        0.000000
통관지세관부호     0.000000
신고인부호       0.000000
수입자부호       0.000101
해외거래처부호     0.340574
특송업체부호      0.672038
수입통관계획코드    0.000000
수입신고구분코드    0.000000
수입거래구분코드    0.000000
수입종류코드      0.000000
징수형태코드      0.000000
신고중량(KG)    0.000000
과세가격원화금액    0.000000
운송수단유형코드    0.000000
반입보세구역부호    0.000000
HS10단위부호    0.000000
적출국가코드      0.000000
원산지국가코드     0.000000
관세율구분코드     0.000000
관세율         0.000000
검사결과코드      0.000000
우범여부        0.000000
핵심적발        0.000000
dtype: float64

In [74]:
df_train['HS10단위부호'] = df_train['HS10단위부호'].astype(str)
df_train['HS_upper'] = df_train['HS10단위부호'].str.slice(start = 0, stop = 2)
df_train['HS_middle'] = df_train['HS10단위부호'].str.slice(start = 2, stop = 4)
df_train['HS_lower'] = df_train['HS10단위부호'].str.slice(start = 4, stop = 6)
df_train['HS_bottom'] = df_train['HS10단위부호'].str.slice(start = 6, stop = 8)
df_train_tmp=df_train.pop('우범여부')
df_train_tmp2=df_train.pop('핵심적발')
df_train['우범여부']=df_train_tmp
df_train['핵심적발']=df_train_tmp2
for data_type in ['우범여부','핵심적발']:
    for i,col in enumerate(list(df_train.columns)[2:-3]):
        print(i,'th saving')
        if len(df_train[col].unique())>100:
            continue
        df_train[[col,data_type]].groupby([col], as_index=True).mean().plot.bar(figsize=(72,15))
        plt.savefig('./{}_{}_dist.jpg'.format(col,data_type))
        plt.clf()
        df_train.groupby(col)[data_type].value_counts(normalize=True).mul(100).rename('percent').reset_index().pipe((sns.catplot,'data'), x=col,y='percent',hue=data_type,kind='bar',size=15)
        plt.savefig('./{}_{}_ratio.jpg'.format(col,data_type))
        # sns.countplot(col,hue='우범여부',data=df_train,ax=ax[1])
        plt.clf()

0 th saving
1 th saving
2 th saving
3 th saving
4 th saving
5 th saving
6 th saving
7 th saving
8 th saving
9 th saving
10 th saving
11 th saving
12 th saving
13 th saving
14 th saving
15 th saving
16 th saving
17 th saving
18 th saving
19 th saving
20 th saving
21 th saving
22 th saving
0 th saving
1 th saving
2 th saving
3 th saving
4 th saving
5 th saving
6 th saving
7 th saving
8 th saving
9 th saving
10 th saving
11 th saving
12 th saving
13 th saving
14 th saving
15 th saving
16 th saving
17 th saving
18 th saving
19 th saving
20 th saving
21 th saving
22 th saving


<Figure size 5184x1080 with 0 Axes>

<Figure size 1194.25x1080 with 0 Axes>

<Figure size 5184x1080 with 0 Axes>

<Figure size 1194.25x1080 with 0 Axes>

...

<Figure size 1194.25x1080 with 0 Axes>

<Figure size 5184x1080 with 0 Axes>

<Figure size 1194.25x1080 with 0 Axes>

<Figure size 5184x1080 with 0 Axes>

<Figure size 1194.25x1080 with 0 Axes>

In [75]:
for data_type in ['우범여부','핵심적발']:
    for col in ['관세율','신고중량(KG)','과세가격원화금액']:
        fig, ax = plt.subplots(1, 1, figsize=(20, 12))
        if data_type=='핵심적발':
            sns.kdeplot(np.log(df_train[df_train[data_type] == 1][col]+1), ax=ax)
            sns.kdeplot(np.log(df_train[df_train[data_type] == 2][col]+1), ax=ax)
            plt.legend(['{} == 0'.format(data_type),'{} == 1'.format(data_type), '{} == 2'.format(data_type)])
        else:
            sns.kdeplot(np.log(df_train[df_train[data_type] == 0][col]+1), ax=ax)
            sns.kdeplot(np.log(df_train[df_train[data_type] == 1][col]+1), ax=ax)
            plt.legend(['{} == 0'.format(data_type),'{} == 1'.format(data_type)])
        plt.savefig('{}_his_{}.jpg'.format(col,data_type))
        plt.clf()

<Figure size 2880x864 with 0 Axes>

<Figure size 2880x864 with 0 Axes>

<Figure size 2880x864 with 0 Axes>

<Figure size 2880x864 with 0 Axes>

<Figure size 2880x864 with 0 Axes>

<Figure size 2880x864 with 0 Axes>